### 라이브러리 Import

In [2]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Optional, Any

from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

from dotenv import load_dotenv
load_dotenv()

True

### 시스템 설정

In [3]:
# Chroma
CHROMA_DB_PATH = "./chroma_db" 

# Neo4j
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")

# OpenAI
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

### PROMPT TEMPLATE
- RAG
- HYBRID

In [4]:
RAG_PROMPT_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 전문가를 위한 유용한 AI 어시스턴트입니다.
제공된 Context (검색된 문서)만을 사용하여 사용자의 질문에 답변하십시오.
만약 Context에 답변할 수 있는 정보가 없다면, '제공된 정보로는 답변할 수 없습니다.'라고 명확하게 언급하세요.

[질문]
{question}

[Context (검색된 문서)]
{context}
"""

In [5]:
HYBRID_QA_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 분석 전문가입니다.
아래에 제공된 두 가지 검색 결과 (Graph DB Context 및 Vector DB Context)를 모두 활용하여 사용자의 질문에 대한 가장 정확하고 포괄적인 답변을 한국어로 생성하십시오.

[질문]
{question}

--- Graph DB Context ---
{graph_context}
---------------------------------------

--- Vector DB Context ---
{vector_context}
-----------------------------------------

# 응답 가이드라인:
- 두 Context를 종합하여 답변하며, 충돌하는 정보가 있을 경우 Graph DB의 사실 정보를 우선하세요.
- 답변은 전문적이고 유익한 톤을 사용하세요.
- 두 Context 모두에 답변할 정보가 없는 경우, '제공된 정보로는 답변을 통합할 수 없습니다.'라고 명확히 언급하세요.
"""

In [7]:
HYBRID_QA_PROMPT = ChatPromptTemplate.from_template(HYBRID_QA_TEMPLATE)

### RAG

In [ ]:
def initialize_chroma_db(embedding_function: OpenAIEmbeddings, collection_name: str) -> Optional[Chroma]:
    """ ChromaDB 벡터 스토어를 초기화합니다. """
    try:
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        # print(f"연결된 컬렉션({collection_name}) 문서 개수: {getattr(vector_db, '_collection').count()}")
        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None
    
def initialize_neo4j_graph() -> Optional[Neo4jGraph]:
    """ Neo4jGraph 객체를 초기화합니다. """
    try:
        graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)
        return graph
    except Exception as e:
        print(f"Neo4j 초기화 오류: {e}")
        return None

In [9]:
def fetch_vector_context(question: str, llm: ChatOpenAI) -> str:
    """ Vector DB에서 관련 문서의 Context를 검색합니다. """
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vector_db = initialize_chroma_db(embeddings, collection_name="unified_data")
        
        if vector_db is None:
            return "ERROR: Vector DB 초기화 실패"
        
        # 5개 문서를 검색하여 텍스트 Context로 변환
        retriever = vector_db.as_retriever(search_kwargs={"k": 5})
        prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
        
        rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
        
        
        docs = retriever.invoke(question)
        
        # 검색된 문서 내용을 하나의 문자열로 결합
        context = "\n---\n".join([doc.page_content for doc in docs])
        
        result = rag_chain.invoke(question)
        
        return context, result
        
    except Exception as e:
        return f"ERROR: Vector DB 검색 오류: {e}"

---

### Graph RAG TEST - LLM으로 Cypher 쿼리 생성 후 쿼리 실행

In [13]:
def generate_cypher_query_and_execute(question):
    graph = initialize_neo4j_graph()
    llm = ChatOpenAI(model_name="gpt-5-mini")
    
    # 스키마 정보 가져오기
    schema = graph.schema
    
    # Cypher 쿼리 생성 프롬프트
    cypher_prompt = f"""
    Given the following Neo4j schema:
    {schema}
    Generate a Cypher query to answer this question: {question}
    Return only the Cypher query, no explanations.
    """
    
    # Cypher 쿼리 생성
    cypher_query = llm.predict(cypher_prompt).strip()
    
    # Cypher 쿼리 실행
    query_result = graph.query(cypher_query)
    return cypher_query, query_result

In [12]:
question = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴와 메뉴 카테고리는 무엇인가요?"

In [17]:
cypher_query, query_result = generate_cypher_query_and_execute(question)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (customers) { ... }} {position: line: 4, column: 1, offset: 193} for query: "MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4\nWITH collect(DISTINCT c) AS customers\nCALL {\n  WITH customers\n  UNWIND customers AS customer\n  MATCH (p:Purchase)-[:BY_CUSTOMER]->(customer)\n  MATCH (p)-[:FOR_MENU_ITEM]->(m:MenuItem)\n  WHERE p.purchaseDate >= date('2024-01-01') AND p.purchaseDate <= date('2024-06-30')\n  RETURN m.name AS menuItem, count(*) AS itemCount\n  ORDER BY itemCount DESC\n  LIMIT 1\n}\nCALL {\n  WITH customers\n  UNWIND customers AS customer\n  MATCH (p:Purchase)-[:

In [18]:
cypher_query

"MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4\nWITH collect(DISTINCT c) AS customers\nCALL {\n  WITH customers\n  UNWIND customers AS customer\n  MATCH (p:Purchase)-[:BY_CUSTOMER]->(customer)\n  MATCH (p)-[:FOR_MENU_ITEM]->(m:MenuItem)\n  WHERE p.purchaseDate >= date('2024-01-01') AND p.purchaseDate <= date('2024-06-30')\n  RETURN m.name AS menuItem, count(*) AS itemCount\n  ORDER BY itemCount DESC\n  LIMIT 1\n}\nCALL {\n  WITH customers\n  UNWIND customers AS customer\n  MATCH (p:Purchase)-[:BY_CUSTOMER]->(customer)\n  MATCH (p)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(mc:MenuCategory)\n  WHERE p.purchaseDate >= date('2024-01-01') AND p.purchaseDate <= date('2024-06-30')\n  RETURN mc.name AS menuCategory, count(*) AS categoryCount\n  ORDER BY categoryCount DESC\n  LIMIT 1\n}\nRETURN menuItem, itemCount, menuCategory, categoryCount;"

In [19]:
query_result

[{'menuItem': '티라미수',
  'itemCount': 36,
  'menuCategory': '디저트',
  'categoryCount': 36}]

---

### LLM 사용 - 복잡한 사용자 질문 분해
- 복잡한 사용자 질문을 '그래프 DB조회'에 필요한 단순한 탐색 경로/중간 데이터 보조 질문으로 분해

In [27]:
import json
from typing import List, Dict, Tuple, Any

def decompose_question_to_sub_queries(question: str, schema: str, llm: ChatOpenAI) -> List[str]:
    
    """
    LLM을 사용하여 복잡한 사용자 질문을 '그래프 DB 조회'에 필요한
    단순한 탐색 경로/중간 데이터 보조 질문으로 분해합니다.
    """
    decompose_prompt = f"""
    You are an expert Graph Query Decomposer. Your task is to analyze the user's complex question based ONLY on the provided Neo4j Schema.
    
    Decompose the question into a sequence of *data retrieval* steps (sub-questions). Each sub-question must target a specific piece of information from the graph. DO NOT ask about schema definition, delivery format, or user intent/policy.
    
    Neo4j Schema:
    ---
    {schema} 
    ---
    
    User Question: "{question}"
    
    Example Output: ["What is the name of the Head Chef?", "What menu items did the Head Chef create?", "Which ingredients are contained in those menu items?"]
    
    Return ONLY a JSON array of strings, where each string is a sub-question focused on data retrieval.
    """
    
    try:
        # LLM 호출 및 JSON 파싱
        # LLM의 응답은 JSON 문자열이라고 가정합니다.
        json_str = llm.invoke(decompose_prompt).content.strip()
        sub_questions = json.loads(json_str)
        
        # 리스트가 아니거나 형식이 맞지 않으면 빈 리스트 반환
        if not isinstance(sub_questions, list):
             return []
             
        return sub_questions
        
    except Exception as e:
        print(f"보조 질문 분해 중 오류 발생: {e}")
        return []

In [28]:
graph = initialize_neo4j_graph()
graph.schema
sub_questions = decompose_question_to_sub_queries(question, graph, llm)

In [29]:
sub_questions

['2024-01-01부터 2024-06-30(2024년 상반기) 사이에 평점이 4점 이상인 리뷰들(리뷰 ID, 작성일, 평점)은 무엇인가?',
 '위 리뷰들을 작성한 고객들(고객 ID 또는 고객 노드)은 누구인가?',
 '해당 고객들이 2024-01-01부터 2024-06-30 사이에 생성한 주문들(주문 ID 및 주문일자)은 무엇인가?',
 '위 주문들에 포함된 각 주문 항목(메뉴 아이템)과 각 항목의 주문 수량(또는 주문 건수)은 얼마인가?',
 '각 메뉴 아이템별 총 주문 수량을 집계했을 때 가장 많이 주문된 메뉴 아이템(최다 주문 수)을 가진 메뉴는 무엇인가? (동률이 있으면 모두)',
 '가장 많이 주문된 해당 메뉴 아이템들의 메뉴 카테고리(카테고리명)는 무엇인가?']

In [ ]:
# 보조 질문 실행 및 결과 context 생성 함수
def execute_sub_queries(question: str, sub_queries: List[str], graph: Neo4jGraph, llm: ChatOpenAI) -> Tuple[str, List[Dict[str, Any]]]:
    """
    보조 질문 목록을 순회하며 Cypher 쿼리를 생성 및 실행하고, 모든 결과를 Context로 수집합니다.
    """
    full_schema = graph.schema
    context_results = []
    
    # 각 보조 질문에 대한 쿼리를 생성하고 실행
    for i, sub_q in enumerate(sub_queries):
        # Cypher 쿼리 생성 프롬프트 (전체 스키마 사용)
        cypher_prompt = f"""
        Given the Neo4j schema and the following sub-question from a sequence of questions, generate the necessary Cypher query.
        
        Sub-Question {i+1}: "{sub_q}"
        
        Neo4j Schema:
        ---
        {full_schema}
        ---
        
        Return ONLY the valid Cypher query. Do not include any explanations.
        """
        
        try:
            # Cypher 쿼리 생성
            cypher_query = llm.invoke(cypher_prompt).content.strip()
            
            # Cypher 쿼리 실행
            query_result = graph.query(cypher_query)
            
            # 결과 Context 저장
            context_results.append({
                "sub_question": sub_q,
                "cypher_query": cypher_query,
                "result": query_result
            })
            
        except Exception as e:
            # 쿼리 생성 또는 실행 오류 시, 오류 메시지를 Context에 저장하여 LLM에게 전달
            context_results.append({
                "sub_question": sub_q,
                "error": f"Failed to execute query: {e}"
            })
            
    # 최종 답변을 위한 Context 문자열 생성 (프롬프트로 전달될 형태)
    context_string = f"Original User Question: {question}\n\nContextual Data Retrieved:\n"
    for item in context_results:
        if 'error' in item:
            context_string += f"- Q: {item['sub_question']} | Error: {item['error']}\n"
        else:
            # 결과가 너무 길면 잘라서 전달하거나 요약하는 로직을 추가할 수 있습니다.
            context_string += f"- Q: {item['sub_question']} | Result: {item['result']}\n"
            
    return context_string, context_results

In [30]:
# 예시 호출 (실제 실행 시 주석 해제 및 graph, llm 객체 필요)
final_context, detailed_results = execute_sub_queries(question, sub_questions, graph, llm)
print("--- LLM에게 전달할 최종 Context ---")
print(final_context)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 2, column: 17, offset: 68} for query: 'MATCH (:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nRETURN DISTINCT id(c) AS customerId, c'


--- LLM에게 전달할 최종 Context ---
Original User Question: 2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴와 메뉴 카테고리는 무엇인가요?

Contextual Data Retrieved:
- Q: 2024-01-01부터 2024-06-30(2024년 상반기) 사이에 평점이 4점 이상인 리뷰들(리뷰 ID, 작성일, 평점)은 무엇인가? | Result: [{'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 5), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 6), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 5}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 11), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 17), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 17), 'rating': 5}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 24), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 1, 27), 'rating': 4}, {'reviewId': 1, 'reviewDate': neo4j.time.Date(2024, 2, 10), 'rating': 5}, {'reviewId': 1, 're

In [31]:
# 'sub_question'과 'cypher_query'만 정리해서 출력
summary_for_llm = [
    {
        "sub_question": item.get("sub_question"),
        "cypher_query": item.get("cypher_query")
    }
    for item in detailed_results
    if "sub_question" in item and "cypher_query" in item
]
summary_for_llm

[{'sub_question': '2024-01-01부터 2024-06-30(2024년 상반기) 사이에 평점이 4점 이상인 리뷰들(리뷰 ID, 작성일, 평점)은 무엇인가?',
  'cypher_query': 'MATCH (r:CustomerReview)\nWHERE r.reviewDate >= date("2024-01-01")\n  AND r.reviewDate <= date("2024-06-30")\n  AND r.rating >= 4\nRETURN r.seq AS reviewId, r.reviewDate AS reviewDate, r.rating AS rating\nORDER BY r.reviewDate;'},
 {'sub_question': '위 리뷰들을 작성한 고객들(고객 ID 또는 고객 노드)은 누구인가?',
  'cypher_query': 'MATCH (:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nRETURN DISTINCT id(c) AS customerId, c'},
 {'sub_question': '위 주문들에 포함된 각 주문 항목(메뉴 아이템)과 각 항목의 주문 수량(또는 주문 건수)은 얼마인가?',
  'cypher_query': 'MATCH (p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)\nRETURN m.name AS menuItem, sum(coalesce(p.quantity, 1)) AS totalQuantity, count(p) AS orderCount\nORDER BY totalQuantity DESC;'},
 {'sub_question': '가장 많이 주문된 해당 메뉴 아이템들의 메뉴 카테고리(카테고리명)는 무엇인가?',
  'cypher_query': 'MATCH (p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(c:MenuCategory)\nWITH m, c, sum(p.quantity) A

### 보조 질문, 그에 대한 사이퍼 쿼리 -> 프롬프트 입력

In [32]:
def generate_cypher_query(question, summary_for_llm):
    """
    Cypher 쿼리를 생성하는 함수
    """
    llm = ChatOpenAI(model_name="gpt-5-mini")
    cypher_prompt = f"""
    아래는 참고할 수 있는 서브질문과 Cypher 쿼리 목록입니다:
    {summary_for_llm}
    
    위 정보를 참고하여, 다음 질문에 답할 수 있는 Cypher 쿼리를 생성하세요:
    {question}
    
    설명 없이 Cypher 쿼리만 반환하세요.
    """
    cypher_query = llm.predict(cypher_prompt).strip()
    
    query_result = graph.query(cypher_query)
    return cypher_query, query_result

In [33]:
cypher_query, query_result = generate_cypher_query(question, summary_for_llm)

In [34]:
cypher_query

'MATCH (r:CustomerReview)-[:WRITTEN_BY]->(cust:Customer)\nWHERE r.reviewDate >= date("2024-01-01")\n  AND r.reviewDate <= date("2024-06-30")\n  AND r.rating >= 4\nWITH DISTINCT cust\nMATCH (cust)-[*1..2]-(p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(cat:MenuCategory)\nWITH m, cat, sum(coalesce(p.quantity, 1)) AS totalOrdered\nWITH max(totalOrdered) AS maxOrdered, collect({m:m, cat:cat, total:totalOrdered}) AS items\nUNWIND items AS it\nWITH it, maxOrdered\nWHERE it.total = maxOrdered\nRETURN it.m.name AS menuName, it.cat.name AS categoryName, it.total AS totalOrdered\nORDER BY menuName;'

In [35]:
query_result

[{'menuName': '까르보나라 파스타', 'categoryName': '파스타', 'totalOrdered': 139}]

In [36]:
graph_context = {
    "cypher_query": cypher_query,
    "query_result": query_result
}
graph_context

{'cypher_query': 'MATCH (r:CustomerReview)-[:WRITTEN_BY]->(cust:Customer)\nWHERE r.reviewDate >= date("2024-01-01")\n  AND r.reviewDate <= date("2024-06-30")\n  AND r.rating >= 4\nWITH DISTINCT cust\nMATCH (cust)-[*1..2]-(p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(cat:MenuCategory)\nWITH m, cat, sum(coalesce(p.quantity, 1)) AS totalOrdered\nWITH max(totalOrdered) AS maxOrdered, collect({m:m, cat:cat, total:totalOrdered}) AS items\nUNWIND items AS it\nWITH it, maxOrdered\nWHERE it.total = maxOrdered\nRETURN it.m.name AS menuName, it.cat.name AS categoryName, it.total AS totalOrdered\nORDER BY menuName;',
 'query_result': [{'menuName': '까르보나라 파스타',
   'categoryName': '파스타',
   'totalOrdered': 139}]}

---

In [ ]:
# 하이브리드 파이프라인 

def hybrid_rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Graph DB와 Vector DB의 Context를 결합하여 최종 답변을 생성합니다.
    """
    print("--- 1. Graph DB 컨텍스트 검색 시작 ---")
    # (a) Graph DB 검색 
    graph = initialize_neo4j_graph()
    sub_questions = decompose_question_to_sub_queries(question, graph, llm)
    final_context, detailed_results = execute_sub_queries(question, sub_questions, graph, llm)

    # 'sub_question'과 'cypher_query'만 정리해서 출력
    summary_for_llm = [
        {
            "sub_question": item.get("sub_question"),
            "cypher_query": item.get("cypher_query")
        }
        for item in detailed_results
        if "sub_question" in item and "cypher_query" in item
    ]

    cypher_query, query_result = generate_cypher_query(question, summary_for_llm)
    
    graph_context = {
        "cypher_query": cypher_query,
        "query_result": query_result
    }
    
    print(f"Graph Context: {graph_context}...")

    print("--- 2. Vector DB 컨텍스트 검색 시작 ---")
    # (b) Vector DB 검색 
    vector_context, _ = fetch_vector_context(question, llm)
    print(f"Vector Context: {vector_context}...")

    # 3. Context 결합 및 최종 답변 생성 (LCEL 사용)
    
    # Context를 결합하는 Runnable
    combined_context_runnable = RunnableLambda(
        lambda q: {
            "question": q, 
            "graph_context": graph_context, 
            "vector_context": vector_context
        }
    )
    
    # LCEL 체인 구성: 입력 -> Context 결합 -> Prompt -> LLM -> 결과 파싱
    hybrid_chain = (
        combined_context_runnable
        | HYBRID_QA_PROMPT
        | llm
        | StrOutputParser()
    )
    
    print("--- 3. 하이브리드 LLM 추론 시작 ---")
    try:
        final_answer = hybrid_chain.invoke(question)
        return final_answer
    except Exception as e:
        return f"ERROR: 하이브리드 LLM 추론 오류: {e}"

In [ ]:
# 실행 예시 (사용자가 직접 실행)
question_hybrid = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?"

llm_instance = ChatOpenAI(model_name="gpt-5-mini", temperature=0) 
answer = hybrid_rag_pipeline(question_hybrid, llm_instance)
print("\n=== 최종 하이브리드 RAG 답변 ===")
print(answer)

--- 1. Graph DB 컨텍스트 검색 시작 ---


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_ITEM)} {position: line: 6, column: 53, offset: 210} for query: 'MATCH (c:Customer)-[:WROTE]->(r:Review)\nWHERE r.rating >= 4\n  AND date(r.date) >= date("2024-01-01")\n  AND date(r.date) <= date("2024-06-30")\nWITH DISTINCT c\nMATCH (c)-[:PLACED|:ORDERED]->(o:Order)-[:CONTAINS|:HAS_ITEM]->(item)-[:BELONGS_TO|:IN_CATEGORY]->(cat:Category)\nRETURN cat.name AS category, count(*) AS order_count\nORDER BY order_count DESC\nLIMIT 1;'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.Unkn

Graph Context: {'cypher_query': 'MATCH (c:Customer)-[:WROTE]->(r:Review)\nWHERE r.rating >= 4\n  AND date(r.date) >= date("2024-01-01")\n  AND date(r.date) <= date("2024-06-30")\nWITH DISTINCT c\nMATCH (c)-[:PLACED|:ORDERED]->(o:Order)-[:CONTAINS|:HAS_ITEM]->(item)-[:BELONGS_TO|:IN_CATEGORY]->(cat:Category)\nRETURN cat.name AS category, count(*) AS order_count\nORDER BY order_count DESC\nLIMIT 1;', 'query_result': []}...
--- 2. Vector DB 컨텍스트 검색 시작 ---


C:\Users\faeqs\AppData\Local\Temp\ipykernel_31728\1920904252.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


Vector Context: 사용자 사용자43이 2024-04-30에 평점 4점으로 '가격은 조금 비싸지만 서비스는 만족스러워요.'라는 리뷰를 남겼습니다.
---
사용자 사용자34이 2024-10-26에 평점 4점으로 '직원분들이 친절해서 기분 좋게 식사했습니다.'라는 리뷰를 남겼습니다.
---
사용자 사용자4이 2024-09-22에 평점 3점으로 '가격은 조금 비싸지만 서비스는 만족스러워요.'라는 리뷰를 남겼습니다.
---
사용자 사용자34이 2024-09-25에 평점 3점으로 '가격은 조금 비싸지만 서비스는 만족스러워요.'라는 리뷰를 남겼습니다.
---
사용자 사용자4이 2024-12-29에 평점 5점으로 '커피가 진하고 맛있습니다.'라는 리뷰를 남겼습니다....
--- 3. 하이브리드 LLM 추론 시작 ---

=== 최종 하이브리드 RAG 답변 ===
제공된 정보로는 답변을 통합할 수 없습니다.

이유 요약
- Graph DB 컨텍스트에서 실행된 Cypher 쿼리는 결과가 비어 있었습니다([]). Graph DB의 사실을 우선하기 때문에, 해당 쿼리 결과가 “가장 많이 주문한 메뉴 카테고리”를 판단할 근거가 되지 않습니다.
- Vector DB 컨텍스트에는 2024년 상반기(1~6월) 조건에 부합하는 긍정 리뷰(평점 ≥ 4)가 사용자43의 2024-04-30 리뷰 하나만 기록되어 있으나, 이 항목에는 해당 사용자의 주문 내역이나 주문한 메뉴 카테고리 정보가 포함되어 있지 않습니다.

따라서 현재 제공된 자료만으로는 “2024년 상반기에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리”를 결정할 수 없습니다.

권장 조치 (데이터 확인 및 재분석 방법)
1. Graph DB 데이터 확인
   - 리뷰 노드(r.date)의 날짜 형식이 date 타입인지(문자열이면 변환 필요)와 r.rating이 숫자 타입인지 확인하세요.
   - 고객(Customer)과 주문(Order), 주문항목(또는 Item), 카테고리(Category) 사